# Mapa do resultado das Eleições de Deputado Estatudais de 2018 por Zona Eleitoral

In [ ]:
'''
Fontes:
IBGE - shapefiles
https://mapas.ibge.gov.br/bases-e-referenciais/bases-cartograficas/malhas-digitais.html
ftp://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2015/Brasil/BR/br_municipios.zip

TSE - resultado eleição
http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais/
http://agencia.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2018.zip
http://www.tse.jus.br/eleitor/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais

mapaslivres - correspondencia entre TSE e IBGE
https://github.com/mapaslivres/zonas-eleitorais

Estadão Dados - zonas eleitorais
https://drive.google.com/drive/u/1/folders/1qwTlkNnt5SEnzirhrXbSXGST6vNvUAVM

Dados Abertos SP - shapefile
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/distritos/resource/9e75c2f7-5729-4398-8a83-b4640f072b5d
'''

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
df_chunk = pd.read_csv('./votos/votacao_candidato_munzona_2018_SP.csv', sep=';', encoding='latin1', chunksize=10000)

In [ ]:
cols = ['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'DS_CARGO',
        'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO', 
        'QT_VOTOS_NOMINAIS']

DS_CARGO = 'Deputado Estadual'
NR_CANDIDATO = 25555 

In [ ]:
#df = df_chunk.get_chunk(5)
#df = df[df['DS_CARGO'] == DS_CARGO]
#df = df[df['NR_CANDIDATO'] == NR_CANDIDATO]
#df = df[cols]
#df.head().T

In [ ]:
chunk_list = []

# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = chunk[chunk['DS_CARGO'] == DS_CARGO]
    #chunk_filter = chunk_filter[chunk_filter['NR_CANDIDATO'] == NR_CANDIDATO]
    chunk_filter = chunk_filter[cols]
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

In [ ]:
df_concat.head()

In [ ]:
df_candidato = df_concat[df_concat['NR_CANDIDATO'] == NR_CANDIDATO]

In [ ]:
df_zona = df_candidato.groupby(['NR_ZONA'])['QT_VOTOS_NOMINAIS'].sum().reset_index()

In [ ]:
df_zona['QT_VOTOS_NOMINAIS'].sum()

In [ ]:
df_mapa = gpd.read_file('./geo/SP_ZONAS_janeiro_2018/ZONAS_FINAL.shp')

In [ ]:
df_mapa_novo = df_mapa.join(df_zona)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, figsize=(12,12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
ax = df_mapa_novo.plot(column='QT_VOTOS_NOMINAIS', legend=True,
                       legend_kwds={'label': "Votos", 'orientation': "vertical"}, 
                       linewidth=0.1, ax=ax, cax=cax, edgecolor='grey')
ax.set_axis_off()
ax.set_title('Resultado Eleição 2018 por Município de São Paulo', fontdict={'fontsize':'25', 'fontweight':'3'})
ax.annotate('Fonte: Estadão Dados, 2018; TSE, 2018', xy=(0.1,0.08),
            xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=10, color='#555555')
plt.show()